In [11]:
import pandas as pd
import numpy as np

from collections import Counter
from itertools import product, combinations

The equation becomes $\frac{1}{x} + \frac{1}{y} = \frac{1}{n} \iff \frac{n}{x} + \frac{n}{y} = 1 \iff nx + ny = xy$. Some observations, given $n$:
1. Since $x,y$ are positive integers, $nx < xy \implies n < y$, and similarly $n < x$. So $n < \min(x,y)$.
2. If $x, y, n$ is a solution, then $y, x, n$ is a duplicate solution. So we can just check ones where $x \leq y$. Combining this with step $1$, we can only check solutions where $n < x \leq y$.
3. If $n < x \leq y$, if $y < 2n$, then $\frac{n}{x} + \frac{n}{y} \geq \frac{2n}{y} > 1$. This means we have to have $y \geq 2n$.
4. If we choose an $x$, we have a closed form expression for $y$:
$$\frac{1}{x} + \frac{1}{y} = \frac{1}{n} \implies y = \frac{nx}{x - n} \implies \frac{\text{d}y}{\text{d}x} = -\frac{n}{(x - n)^2} < 0$$
Since $y$ is an integer, the right hand side numerator and denominator must divide. The derivative also shows when $x$ increases, $y$ decreases. Since there is a lower bound on $y$ (observation 3), there must be an implied upper bound on $x$. In particular, if $y = 2n$ (extreme lower bound) then $x = 2n$ (extreme upper bound), so $x \leq 2n$.

---

Suppose $x = n + a$ and $y = n + b$ for $a, b \in \Z$. Obvsiouly, $a,b > 1$, due to observation $1$ above. If we perform the math, we get
$$
\begin{align*}
&\frac{1}{n+a} + \frac{1}{n+b} = \frac{1}{n} \\
\implies &\frac{a + b + 2n}{(n+a)(n+b)} = \frac{1}{n} \\
\implies &an + bn + 2n^2 = n^2 + an + bn + ab \\
\implies &n^2 = ab.
\end{align*}
$$

Thus for each $n$, we just need for find each divisor of $n^2$ and that will give us each $a$ (and $b$ will come as a pair). To speed it up even further, we can use prime factorization.

Suppose $n^2 = \prod_{i=1}^{k} p_i^{k_i}$ where $p_k$ is the largest prime factor of $n$. Then the number of factors is $\prod_{i=1}^{k} (k_i + 1)$, since for each $p_i$ we can choose to set the exponent to $0, 1, 2, \dots, k_i$, which is $k_i + 1$ possible values for each prime. Additionally, we need to divide by $2$ and add $1$ to account for duplicate values.


In [50]:
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

def prime_factorize(num, primes = None):
    prime_factors = []
    n = num
    for p in primes:
        if n == 1 or p > n or (len(prime_factors) == 0 and p >= 100):
            break
        while n % p == 0:
            prime_factors.append(p)
            n //= p

    return Counter(prime_factors)


In [49]:
# second attempt
pbs, ps = sieve(10**6)
n = 4
max_cnt = -1
max_n = -1
while True:
    vals = prime_factorize(n*n, ps).values()
    cnt = 1
    for val in vals:
        cnt *= val+1
    
    # reduce by duplicate values
    cnt = (cnt // 2) + 1

    if cnt > max_cnt:
        print(n, cnt)
        max_cnt = cnt
        max_n = n

    # print(n, cnt)
    if cnt > 1000:
        print(n)
        break

    n += 1

4 3
6 5
12 8
24 11
30 14
60 23
120 32
180 38
210 41
360 53
420 68
840 95
1260 113
1680 122
2520 158
4620 203
7560 221
9240 284
13860 338
18480 365
27720 473
55440 608
83160 662
110880 743
120120 851
180180 1013
180180


In [9]:
# first attempt
n = 4
max_cnt = -1
max_n = -1
while True:
    xs = np.arange(n+1, 2*n + 1, dtype = int)
    cnt = np.sum(n*xs % (xs - n) == 0)

    if cnt > max_cnt:
        print(n, cnt)
        print(1*(n*xs % (xs - n) == 0))
        print()
        max_cnt = cnt
        max_n = n

    # print(n, cnt)
    if cnt > 1000:
        print(n)
        break

    n += 1

4 3
[1 1 0 1]

6 5
[1 1 1 1 0 1]

12 8
[1 1 1 1 0 1 0 1 1 0 0 1]

24 11
[1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1]

30 14
[1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1]

60 23
[1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1]

120 32
[1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0
 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]

180 38
[1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

210 41
[1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0

KeyboardInterrupt: 